In [1]:
import xlwings as xw
import os
import re

In [2]:
# Input file paths
lokesh_file = "./data/lokesh/Fall2019_ASTR116-10_Essay1_anonymized_Aspirational_tacited.xlsx"
reina_file = "./data/reina/Fall2019_ASTR116-10_Essay1_anonymized_Aspirational_tacited_reina.xlsx"

### Reconciliing Annotations: 4 Cases

1. Case that Lokesh and Reina disagrees
    * One of them has a 0 ("No")
2. Case that Lokesh and Reina agrees (negative theme)
    * They have the same annotation, which is 0.
3. Case that Lokesh annd Reina agrees (positive theme) AND have similar highlighted text
    * They have the same annotation and the same red-only annotation.
4. Case that Lokesh and Reina agrees (postiive theme) BUT have different highlighted text
    * They have the same "Yes" labels, but have different highlighted text -> merge_stings

In [3]:
def process_reconciled_labels(k_col, l_col, m_col):
    """
    Reconciling labels
    input: 
    * reina's labels -> k_col
    * lokesh's lables -> l_col
    * reconciled column range -> m_col
    """
    print("Processing labels...")
    for k_cell, l_cell, m_cell in zip(k_col[1:], l_col[1:], m_col):
        # if reina's labels match lokesh's labels
        if k_cell.value == l_cell.value:
            # the reconciled value is the same matching value
            m_cell.value = k_cell.value
        # if not
        else:
            # then the reconciled value is undetermined 
            m_cell.value = "X"
            m_cell.font.color = (0,0,0)
            m_cell.color = (255, 0, 0)  # Set font color to red
    print("Done!")

def merge_strings(a, b):
    """
    Helper function to merge two similar strings
    To be used for Case 4
    """
    # Split strings into lists of words
    words_a = a.split()
    words_b = b.split()

    # Find the common prefix
    common_prefix = []
    for word_a, word_b in zip(words_a, words_b):
        if word_a == word_b:
            common_prefix.append(word_a)
        else:
            break

    # Join the common prefix and the remaining words from b
    merged_string = " ".join(common_prefix + words_b[len(common_prefix):])

    return merged_string


# merged_string = merge_strings(cell_n15, cell_o15)

# #Return the string to P
# print(merged_string)

def process_reconciled_annotations(n_col, o_col, p_col, q_col, r_col, combined_sheet):
    """
    Reconciling annotations
    input: 
    * reina's annotations -> n_col
    * lokesh's annotations -> o_col
    * reina's red-only annotations -> p_col
    * lokesh's red-only annotations -> q_col
    * reconciled column r range -> r_col
    """
    print("Processing annotations...")

    # For each value in n_col, o_col, p_col, q_col, r_col
    for n_cell, o_cell, p_cell, q_cell, r_cell in zip(n_col[1:], o_col[1:], p_col, q_col, r_col):

        # Get the cell addresses of Reina and Lokesh's annotations 
        n_cell_address = n_cell.address # The address of Reina's annotation cell ex. N1
        o_cell_address = o_cell.address # Address of Lokesh's annotation cell ex. O1
        p_cell_address = p_cell.address # Address of cell at column P, this is where we will put only the red annotations of Reina
        q_cell_address = q_cell.address # Address of cell at column Q, this is where we will put only the red annotations of Lokesh
        r_cell_address = r_cell.address # Address of cell at column R, this is where we will put the reconciled red text from Reina and Lokesh

        # Pass these cell addresses to get_color_text(x)
        n_cell_color_only = f"=IF(GetColorText({n_cell_address})=\"\",0,GetColorText({n_cell_address}))"
        o_cell_color_only = f"=IF(GetColorText({o_cell_address})=\"\",0,GetColorText({o_cell_address}))"

        combined_sheet.range(p_cell_address).value = n_cell_color_only
        combined_sheet.range(q_cell_address).value = o_cell_color_only

        # print(n_cell_color_only)

        p_cell_value = combined_sheet.range(p_cell_address).value
        q_cell_value = combined_sheet.range(q_cell_address).value
        r_cell_value = combined_sheet.range(r_cell_address).value

        # print(p_cell_value)
        # print(q_cell_value)
        # print(r_cell_value)
      
        # Case 2 and 3
        if (p_cell_value == q_cell_value): # Return 0 or that red-only text
            r_cell.value = p_cell_value
        # Case 4:
        elif (p_cell_value != q_cell_value) and p_cell_value != 0 and q_cell_value != 0:   
            r_cell.value = merge_strings(p_cell_value, q_cell_value)
        # Case 1: 
        else:
            # r_cell value will n_cell or o_cell depending if whether it's a string or not
            r_cell.value = n_cell.value if isinstance(p_cell.value, str) else o_cell.value if isinstance(q_cell.value, str) else 0
            r_cell.font.color = (0,0,0)
            r_cell.color = (255,255,0)  # Set cell color to yellow

    print("Done!")

In [4]:
def reconcile(book_1_path, book_2_path, out_path):
    # Load both workbooks
    print("Starting reconciliation")
    print("Loading workbooks...")

    base_name_1 = os.path.basename(book_1_path)
    base_name_2 = os.path.basename(book_2_path)

    dir_name_1 = os.path.dirname(book_1_path)
    dir_name_2 = os.path.dirname(book_2_path)

    common_name = ""
   

    if base_name_1 == base_name_2:
        # print("Renaming files first.")
        common_name = base_name_1[:-5]
        # Specify the new names based on folder
        new_name1 = os.path.join(dir_name_1, "Anon1_"+ base_name_1) 
        new_name2 = os.path.join(dir_name_2, "Anon2_" + base_name_1)

        # Rename the files
        os.rename(os.path.join(dir_name_1, base_name_1), new_name1)
        os.rename(os.path.join(dir_name_2, base_name_2), new_name2)

        new_book_1_path = os.path.join(dir_name_1, new_name1)
        new_book_2_path = os.path.join(dir_name_2, new_name2)

        # print("Naming successful")

       

        wb_1 = xw.Book(new_book_1_path) # Reina
        wb_2 = xw.Book(new_book_2_path) # Lokesh
    else:
        replace_text_pattern = r'(RA_|_ra|ra_|Reina_|_reina|Lokesh_|_lokesh|Anon1_|Anon2_)'
        common_name = re.sub(replace_text_pattern, "", os.path.splitext(base_name_1[:-5])[0])  
        wb_1 = xw.Book(book_1_path) # Reina
        wb_2 = xw.Book(book_2_path) # Lokesh

   
    # Get the sheets from both workbooks
    sheet_1 = wb_1.sheets.active
    sheet_2 = wb_2.sheets.active

    # Combined sheet
    wb_3 = xw.Book()
    sheet_3 = wb_3.sheets.active

    # Renaming columns
    # Reina's sheets
    sheet_1.range("K1").value = "Reina Aspirational Present"
    sheet_1.range("L1").value = "Reina Annotated Essay"
    sheet_1.range("M1").value = "Reina Comments"

    # Lokesh's sheets
    sheet_2.range("K1").value = "Lokesh Aspirational Present"
    sheet_2.range("L1").value = "Lokesh Annotated Essay"
    sheet_2.range("M1").value = "Lokesh Comments"

    # Combined sheet
    sheet_3.range("K1").value = "Reina Label"
    sheet_3.range("L1").value = "Lokesh Label"
    sheet_3.range("M1").value = "Reconciled"
    sheet_3.range("P1").value = "Reina Red-only Annotation"
    sheet_3.range("Q1").value = "Lokesh Red-only Annotation"
    sheet_3.range("R1").value = "Reconciled Red-only Annotation"
    
    # Formatting
    sheet_3.range('M1').font.bold = True
    sheet_3.range('P1').font.bold = True
    sheet_3.range('Q1').font.bold = True
    sheet_3.range('R1').font.bold = True
    sheet_3.range('R1').WrapText = True

    # Populating combined sheet
    # Retrieving labels ("Yes" or "No") from Reina and Lokesh's sheets
    reina_labels = sheet_1.range("K1").expand("down")
    lokesh_labels = sheet_2.range("K1").expand("down")
    reina_annotations = sheet_1.range("L1").expand("down") 
    lokesh_annotations = sheet_2.range("L1").expand("down")

    # Copy over information from Reina and Lokesh's workbooks to the combined sheet
    sheet_1.range('A1:K1').expand("down").copy(sheet_3.range('A1:K1'))
    sheet_2.range('K1').expand("down").copy(sheet_3.range("L1"))
    reina_annotations.copy(sheet_3.range("N1"))
    lokesh_annotations.copy(sheet_3.range("O1"))

    # Initiate the column variables we will be working with in combined sheet
    reina_labels = sheet_3.range("K1").expand("down")
    lokesh_labels = sheet_3.range("L1").expand("down")
    reconciled_labels = sheet_3.range("M:M")[1:] # M column, this is where we will put all of the reconciled labels
    
    reina_annotations = sheet_3.range("N1").expand("down") # N column
    lokesh_annotations = sheet_3.range("O1").expand("down") # O column
    column_p_range = sheet_3.range("P:P")[1:] # P column, Reina's red-only text
    column_q_range = sheet_3.range("Q:Q")[1:] # Q column, Lokesh's red-only text
    column_r_range = sheet_3.range("R:R")[1:] # R column, Reconciled red text

    # Run functions to process labels and annotations
    process_reconciled_labels(reina_labels, lokesh_labels, reconciled_labels)
    process_reconciled_annotations(reina_annotations, lokesh_annotations, column_p_range, column_q_range, column_r_range, sheet_3)
    

    out_file_name = f"{out_path}/" + common_name + "_reconciled.xlsm"
  

    wb_3.save(out_file_name)
    # print("Reconciliation finished. Closing workbooks.")
    # print(f"Combined sheet in {out_file_name}")
    wb_2.save()
    wb_1.save()
    wb_3.close()
    wb_2.close()
    wb_1.close()
    
    return out_file_name
    



In [5]:
import re
def generate_folder(dir_path):
    """
    Creates hierarchy of directories for the output.
    Follows the structure: [Root Output] > [Semester Year] > [Courese Name and ID] > [Section] > [*.csv]
    """
    folder_dic = {"season": "", "year": "", "course_name": "", "section": ""}
   
    try:
        season = re.search("Fall|Spring|FALL|SPRING", dir_path).group()
        # print(season)
        year = re.search("20\d+", dir_path).group()
        # print(year)
        course_name = re.search("((PHYS|CHEM|MATH|BIO|ASTR|CSC|SCI))\d+", dir_path).group()
        course_name = re.search("^[a-zA-Z]+", course_name).group() + " " + re.search("\d+", course_name).group()
        # print(course_name)
        section = "Section " + re.search("(?<=-)\d+", dir_path).group()
        # print(section)
        folder_dic["season"] = season
        folder_dic["year"] = year
        folder_dic["course_name"] = course_name
        folder_dic["section"] = section
    except:
        print("Error in get_folder_names(). New folder creation unsuccessful.")

    if os.path.isdir(f"/Users/gbaldonado/Library/Containers/com.microsoft.Excel/Data/reconciled/attainment/batch2/{folder_dic['season']} {folder_dic['year']}/{folder_dic['course_name']}/{folder_dic['section']}"):
        print("Folder already exists")
    else:
        os.makedirs(f"/Users/gbaldonado/Library/Containers/com.microsoft.Excel/Data/reconciled/attainment/batch2/{folder_dic['season']} {folder_dic['year']}/{folder_dic['course_name']}/{folder_dic['section']}")
        print(f"Successfully created /Users/gbaldonado/Library/Containers/com.microsoft.Excel/Data/reconciled/aspirational/{folder_dic['season']} {folder_dic['year']}/{folder_dic['course_name']}/{folder_dic['section']} folder")
    
    out_path = f"/Users/gbaldonado/Library/Containers/com.microsoft.Excel/Data/reconciled/attainment/batch2/{folder_dic['season']} {folder_dic['year']}/{folder_dic['course_name']}/{folder_dic['section']}"
    
    return out_path

In [6]:
# Function to filter .xlsx files with "Essay1" in their names
def filter_xlsx_files(folder_list, filter):
    xlsx_files = []
    for folder in folder_list:
        if filter in folder:
            xlsx_files.append(folder)
    return xlsx_files

In [7]:
import glob

def reconciliation_wrapper(folder_1_path, folder_2_path, overwrite=False):
   

    # create a list of inputs
    input_dir_1 = sorted(glob.glob(f"{folder_1_path}/**/*.xlsx", recursive=True))

    # create a list of inputs
    input_dir_2 = sorted(glob.glob(f"{folder_2_path}/**/*.xlsx", recursive=True))

    input_dir_1 = filter_xlsx_files(input_dir_1, "Essay1")
    input_dir_2 = filter_xlsx_files(input_dir_2, "Essay1")

    # print(input_dir_1, len(input_dir_1))
    # print(input_dir_2, len(input_dir_2))

    for i,j in zip(input_dir_1, input_dir_2):
        # print("hello")
        # print("We're here:", str(i), str(j))
        name_i = os.path.basename(i)
        name_j = os.path.basename(j)
        common_name = ""
        if name_i == name_j:
            common_name = name_i[:-5]
        else:
            replace_text_pattern = r'(RA_|_ra|ra_|Reina_|_reina|Lokesh_|_lokesh|Anon1_|Anon2_)'
            common_name = re.sub(replace_text_pattern, "", os.path.splitext(name_i[:-5])[0])
        output_path = generate_folder(i)
        out_file_name_1 = f"{output_path}/" + str(common_name) +"_reconciled.xlsm"

        print("out_file_name_1", out_file_name_1)
        if overwrite == "False" and os.path.isfile(out_file_name) == True:
            print(f"File {common_name} already exists. No overwriting was performed. Terminating process for this file.")
        else:
            out_file_name = reconcile(i, j, output_path)
            print(out_file_name + " reconciled successfully.")

     
    

In [26]:
i = [1,2,3,4]
j = ["A", "B", "C", "D"]

for c,d, in zip(i,j):
    print(str(c) + " " + d)

1 A
2 B
3 C
4 D


In [8]:
reina_folder = "/Users/gbaldonado/Library/Containers/com.microsoft.Excel/Data/reconciled/attainment/batch2/reina_batch_2"
gian_folder = "/Users/gbaldonado/Library/Containers/com.microsoft.Excel/Data/reconciled/attainment/batch2/gian_batch_2"

In [9]:
reconciliation_wrapper(reina_folder, gian_folder)

Successfully created /Users/gbaldonado/Library/Containers/com.microsoft.Excel/Data/reconciled/aspirational/Fall 2020/ASTR 0116/Section 02 folder
out_file_name_1 /Users/gbaldonado/Library/Containers/com.microsoft.Excel/Data/reconciled/attainment/batch2/Fall 2020/ASTR 0116/Section 02/Fall2020_ASTR0116-02_Essay1_anonymized_tacited_reconciled.xlsm
Starting reconciliation
Loading workbooks...
Processing labels...
Done!
Processing annotations...
Done!
/Users/gbaldonado/Library/Containers/com.microsoft.Excel/Data/reconciled/attainment/batch2/Fall 2020/ASTR 0116/Section 02/Fall2020_ASTR0116-02_Essay1_anonymized_tacited_reconciled.xlsm reconciled successfully.
Successfully created /Users/gbaldonado/Library/Containers/com.microsoft.Excel/Data/reconciled/aspirational/Fall 2020/PHYS 0112/Section 01 folder
out_file_name_1 /Users/gbaldonado/Library/Containers/com.microsoft.Excel/Data/reconciled/attainment/batch2/Fall 2020/PHYS 0112/Section 01/Fall2020_PHYS0112-01_Essay1_anonymized_tacited_reconciled

: 

In [ ]:
print("hello")

In [ ]:
a = "/Users/gbaldonado/Developer/TACIT/ALMA-TACIT/gian-work/annotation_notebooks/reconciliation/data/reina/Spring2020_PHYS0102-04_Essay1_anonymized_tacited.xlsx"
b = "/Users/gbaldonado/Developer/TACIT/ALMA-TACIT/gian-work/annotation_notebooks/reconciliation/data/lokesh/Spring2020_PHYS0102-04_Essay1_anonymized_tacited.xlsx"

In [ ]:
print("/Users/gbaldonado/Library/Containers/com.microsoft.Excel/Data")